In [ ]:
import pandas as pd
from rich.live import Live
from rich.table import Table
from rich.panel import Panel
from rich.layout import Layout
import importlib
import re

# Import trading library
from tradeSim import TradeSim
from tradeSim import StrategyHandler

# **__Trading simulation__**

Welcome to the simulation platform!

This Python-based environment is designed to replicate a realistic trading experience using the SET50 daily ticks dataset. It supports essential features like real-time data streaming, order matching, portfolio management, and fee calculations, giving you everything you need to build and test your algorithmic trading strategies.

Each participant begins with a virtual portfolio containing 10,000,000 Baht in starting capital. Your challenge is to design effective trading algorithms and compete with other teams to generate the highest return. Think smart, trade fast, and enjoy the competition!

---

### **Import Daily ticks information for trading infomation**
For each day within compitition, the tick information will be provided daily

In [ ]:
#TODO: Change ticks information daily
daily_ticks = "./marketInfo/ticks/2025-10-24.csv"

df = pd.read_csv(daily_ticks)
df['TradeDateTime'] = pd.to_datetime(df['TradeDateTime'])
grouped = df.groupby('ShareCode')

### **Now, name your team!**

The code section below will create your trading simulation class using the team name you provide.

- Your team name will be set as the `owner` of the `TraderPortfolio` from `trading_sim`.
- The `SimExecution` class will handle order execution for buying stocks in the market.

**Team name requirements:**
- Must use English letters or numbers  
- No spaces allowed  
- Hyphens (`-`) and underscores (`_`) are allowed  
- Must be **30 characters or fewer**

In [ ]:
#TODO: Replace with your team name
team_name = "example"
strategy_name = "Example_strategy"

pattern = r'^[A-Za-z0-9-_]{1,30}$'
if not bool(re.match(pattern, team_name)) or not bool(re.match(pattern, strategy_name)):
    raise ValueError("Team name or strategy name is invalid. Please use only alphanumeric characters, hyphens, and underscores, with a maximum length of 30 characters.")

# Init trade system
trading_Sim = TradeSim.tradeSim(team_name) 
strategy_runner = trading_Sim.get_strategy_runner()

### **Craft Your Strategy**

To simulate your trading strategy within the platform,  
you must create your own strategy class by inheriting from the `strategy_template` class.

Once you've finished writing or are ready to test it,  
run the code below to import your strategy into this notebook.


In [ ]:
try:
    strategy_module = importlib.import_module(f"strategy.{strategy_name}")
    importlib.reload(strategy_module)
except ImportError as e:
    active_strategy_fn = None
    print(f"Error in strategy module: {e}")
    
strategy_class = getattr(strategy_module, strategy_name, None)

----

## **The Simulation**
The simulation models a real-time market environment using a thread handler, where each thread streams data for a specific symbol in the SET50 index, as provided in ```Daily_ticks.CSV```. This section of the code iterates through all entries in the CSV file, applying each competitor’s strategy based on the current market data. During the streaming process, the system attempts to match any pending orders in the order books and updates the market prices in the competitors’ portfolios for any held stocks.
##### *For more detailed explanation please look at document file*
>By setting `with_visual` to `False`, the simulation runs significantly faster.

In [ ]:
#TODO: Change the `with_visual` according to your preference
# True if you want to see the visual simulation  
# False if you want to finish the simulation faster
with_visual = True

In [ ]:
latest_prices = {}

# === Render Helpers ===
def render_market_table():
    table = Table(title="📈 Market Stream")
    table.add_column("Symbol")
    table.add_column("Last Price", justify="right")
    table.add_column("Volume", justify="right")
    table.add_column("Flag", justify="right")

    snapshot = dict(latest_prices)  # copy to avoid mid-update issues

    for sym, data in snapshot.items():
        table.add_row(
            sym,
            f"{data['price']:.2f}",
            str(data["volume"]),
            data["Flag"]
        )
    return table


def render_portfolio_table():
    table = Table(title=f"💼 Portfolio: {strategy_runner.get_owner()}", expand=True)
    table.add_column("Cash Balance", no_wrap=True, width=18)
    table.add_column("Symbol")
    table.add_column("Actual Vol", justify="right")
    table.add_column("Buy Price", justify="right")
    table.add_column("Market Price", justify="right")
    table.add_column("Average Cost", justify="right")
    table.add_column("Amount Cost", justify="right")
    table.add_column("Market Value", justify="right")
    table.add_column("Unrealized", justify="right")
    table.add_column("Unreal. %", justify="right")
    table.add_column("Realized", justify="right")
    table.add_column("Buy Time", justify="center")

    stock_infos = strategy_runner.get_all_stocks_info()
    cash_str = f"{strategy_runner.get_cash_balance():,.2f}"
    cash_displayed = False

    if not stock_infos:
        table.add_row(cash_str, "-", "0", "-", "-", "-", "-", "-", "-", "-", "-", "-")
    else:
        for stock in stock_infos:
            table.add_row(
                cash_str if not cash_displayed else "",
                stock["Symbol"],
                str(stock["Actual Volume"]),
                f"{stock['Buy Price']:.2f}",
                f"{stock['Market Price']:.2f}",
                f"{stock['Average Cost']:.2f}",
                f"{stock['Amount Cost']:.2f}",
                f"{stock['Market Value']:.2f}",
                f"{stock['Unrealized P&L']:.2f}",
                f"{stock['Unrealized %']:.2f}%",
                f"{stock['Realized P&L']:.2f}",
                stock["Buy time"]
            )
            cash_displayed = True

    # Add metrics summary
    table.add_section()
    table.add_row("🔢 Metrics", "", "", "", "", "", "", "", "", "", "", "")
    table.add_row("ROI (%)", f"{strategy_runner.get_roi():.2f}%", "", "", "", "", "", "", "", "", "", "")
    table.add_row("Max Drawdown (%)", f"{strategy_runner.get_max_draw_down():.2f}%", "", "", "", "", "", "", "", "", "", "")
    table.add_row("Win Count", str(strategy_runner.get_number_of_wins()), "", "", "", "", "", "", "", "", "", "")
    table.add_row("Sell Count", str(strategy_runner.get_number_of_sells()), "", "", "", "", "", "", "", "", "", "")
    return table


def render_layout():
    layout = Layout()
    layout.split_column(
        Layout(Panel(render_market_table(), expand=True), name="market"),
        Layout(Panel(render_portfolio_table(), expand=True), name="portfolio")
    )
    return layout


# === Simulation Core ===
handlers = {
    symbol: StrategyHandler.StrategyHandler(strategy_class, strategy_runner)
    for symbol, _ in grouped
}

max_length = max(len(group) for _, group in grouped)
layout = render_layout()

if with_visual:
    with Live(layout, refresh_per_second=2):
        for tick in range(max_length):
            for symbol, data in grouped:
                if tick >= len(data):
                    continue

                row = data.iloc[tick]
                price_update = {row['ShareCode']: row['LastPrice']}

                handler = handlers[symbol]
                handler.process_row(row)

                if not trading_Sim.isOrderbooksEmpty():
                    trading_Sim.isMatch(row)

                trading_Sim.update_market_prices(price_update)

                latest_prices[symbol] = {
                    "price": row['LastPrice'],
                    "volume": row['Volume'],
                    "Flag": row['Flag']
                }

            layout["market"].update(Panel(render_market_table()))
            layout["portfolio"].update(Panel(render_portfolio_table()))

else:
    # Headless (no visual mode)
    for tick in range(max_length):
        for symbol, data in grouped:
            if tick >= len(data):
                continue

            row = data.iloc[tick]
            price_update = {row['ShareCode']: row['LastPrice']}

            handler = handlers[symbol]
            handler.process_row(row)

            if not trading_Sim.isOrderbooksEmpty():
                trading_Sim.isMatch(row)

            trading_Sim.update_market_prices(price_update)

            latest_prices[symbol] = {
                "price": row['LastPrice'],
                "volume": row['Volume'],
                "Flag": row['Flag']
            }

# === Final flush/save ===
layout["market"].update(Panel(render_market_table()))
layout["portfolio"].update(Panel(render_portfolio_table()))

trading_Sim.flushTransactionLog()
trading_Sim.flushErrorLogger()
trading_Sim.create_transaction_summarize(team_name)
trading_Sim.save_portfolio()

trading_date = df['TradeDateTime'].dt.date.iloc[0]
trading_Sim.save_summary_csv(trading_date)

### **Examine Portfolio Information**

You can view portfolio statistics using the provided methods.

For example:
```
strategy_runner.get_all_stock_info() -> Returns a summary of each stock in the portfolio.
strategy_runner.get_portfolio_info() -> Returns a summary of portfolio details such as cash balance, NAV, and ROI.
```
Full documentation is available in the provided PDF.

In [ ]:
portInfo = strategy_runner.get_portfolio_info()
stockInfo = strategy_runner.get_all_stocks_info()
pd.DataFrame([portInfo]).T

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) 
pd.set_option('display.max_rows', None)

df = pd.DataFrame(stockInfo)
print(df)
